In [ ]:
#https://github.com/jaswindersingh2/SPOT-RNA/
#https://tensorboard.dev/#get-started
#!pip install -U tensorboard --user
#/home/ash022/.local/bin/tensorboard dev upload --logdir /mnt/promec-ns9036k/.tools/guiding-cow-tensorboard/20191030-172602/
#https://tensorboard.dev/experiment/PQnexLz9S06F0Yvykilxdw
!pwd
#https://www.youtube.com/watch?v=B4p6gvPs-gM

In [7]:
#!git clone https://github.com/facebookresearch/CrypTen.git
#!cd CrypTen
#!pip install -e .
import torch
import crypten
crypten.init()
x = torch.tensor([1.0, 2.0, 3.0])
x_enc = crypten.cryptensor(x) # encrypt
x_dec = x_enc.get_plain_text() # decrypt
y_enc = crypten.cryptensor([2.0, 3.0, 4.0])
sum_xy = x_enc + y_enc # add encrypted tensors
sum_xy_dec = sum_xy.get_plain_text() # decrypt sum
print(x_dec,y_enc,sum_xy_dec)

tensor([1., 2., 3.]) MPCTensor(
	_tensor=tensor([131072, 196608, 262144])
	plain_text=HIDDEN
	ptype=ptype.arithmetic
) tensor([3., 5., 7.])


# https://github.com/facebookresearch/CrypTen/tree/master/examples/mpc_linear_svm

In [8]:
import logging
import time

import crypten
import torch
from examples.meters import AverageMeter


def train_linear_svm(features, labels, epochs=50, lr=0.5, print_time=False):
    # Initialize random weights
    w = features.new(torch.randn(1, features.size(0)))
    b = features.new(torch.randn(1))

    if print_time:
        pt_time = AverageMeter()
        end = time.time()

    for epoch in range(epochs):
        # Forward
        label_predictions = w.matmul(features).add(b).sign()

        # Compute accuracy
        correct = label_predictions.mul(labels)
        accuracy = correct.add(1).div(2).mean()
        if crypten.is_encrypted_tensor(accuracy):
            accuracy = accuracy.get_plain_text()

        # Print Accuracy once
        if crypten.communicator.get().get_rank() == 0:
            logging.info(
                f"Epoch {epoch} --- Training Accuracy %.2f%%" % (accuracy.item() * 100)
            )

        # Backward
        loss_grad = -labels * (1 - correct) * 0.5  # Hinge loss
        b_grad = loss_grad.mean()
        w_grad = loss_grad.matmul(features.t()).div(loss_grad.size(1))

        # Update
        w -= w_grad * lr
        b -= b_grad * lr

        if print_time:
            iter_time = time.time() - end
            pt_time.add(iter_time)
            logging.info("    Time %.6f (%.6f)" % (iter_time, pt_time.value()))
            end = time.time()

    return w, b


def evaluate_linear_svm(features, labels, w, b):
    """Compute accuracy on a test set"""
    predictions = w.matmul(features).add(b).sign()
    correct = predictions.mul(labels)
    accuracy = correct.add(1).div(2).mean().get_plain_text()
    if crypten.communicator.get().get_rank() == 0:
        print("Test accuracy %.2f%%" % (accuracy.item() * 100))


def run_mpc_linear_svm(
    epochs=50, examples=50, features=100, lr=0.5, skip_plaintext=False
):
    crypten.init()

    # Set random seed for reproducibility
    torch.manual_seed(1)

    # Initialize x, y, w, b
    x = torch.randn(features, examples)
    w_true = torch.randn(1, features)
    b_true = torch.randn(1)
    y = w_true.matmul(x) + b_true
    y = y.sign()

    if not skip_plaintext:
        logging.info("==================")
        logging.info("PyTorch Training")
        logging.info("==================")
        w_torch, b_torch = train_linear_svm(x, y, lr=lr, print_time=True)

    # Encrypt features / labels
    x = crypten.cryptensor(x)
    y = crypten.cryptensor(y)

    logging.info("==================")
    logging.info("CrypTen Training")
    logging.info("==================")
    w, b = train_linear_svm(x, y, lr=lr, print_time=True)

    if not skip_plaintext:
        logging.info("PyTorch Weights  :")
        logging.info(w_torch)
    logging.info("CrypTen Weights:")
    logging.info(w.get_plain_text())

    if not skip_plaintext:
        logging.info("PyTorch Bias  :")
        logging.info(b_torch)
    logging.info("CrypTen Bias:")
    logging.info(b.get_plain_text())

In [ ]:
import argparse
import logging
import os

from examples.multiprocess_launcher import MultiProcessLauncher


parser = argparse.ArgumentParser(description="CrypTen Linear SVM Training")
parser.add_argument(
    "--world_size",
    type=int,
    default=2,
    help="The number of parties to launch. Each party acts as its own process",
)
parser.add_argument(
    "--epochs", default=50, type=int, metavar="N", help="number of total epochs to run"
)
parser.add_argument(
    "--examples", default=50, type=int, metavar="N", help="number of examples per epoch"
)
parser.add_argument(
    "--features",
    default=100,
    type=int,
    metavar="N",
    help="number of features per example",
)
parser.add_argument(
    "--lr", "--learning-rate", default=0.5, type=float, help="initial learning rate"
)
parser.add_argument(
    "--skip_plaintext",
    default=False,
    action="store_true",
    help="skip evaluation for plaintext svm",
)
parser.add_argument(
    "--multiprocess",
    default=False,
    action="store_true",
    help="Run example in multiprocess mode",
)


def _run_experiment(args):
    level = logging.INFO
    if "RANK" in os.environ and os.environ["RANK"] != "0":
        level = logging.CRITICAL
    logging.getLogger().setLevel(level)
    logging.basicConfig(
        level=level,
        format="%(asctime)s - %(process)d - %(name)s - %(levelname)s - %(message)s",
    )
    from mpc_linear_svm import run_mpc_linear_svm

    run_mpc_linear_svm(
        args.epochs, args.examples, args.features, args.lr, args.skip_plaintext
    )


def main(run_experiment):
    args = parser.parse_args()
    if args.multiprocess:
        launcher = MultiProcessLauncher(args.world_size, run_experiment, args)
        launcher.start()
        launcher.join()
        launcher.terminate()
    else:
        run_experiment(args)


if __name__ == "__main__":
    main(_run_experiment)

In [ ]:
%%bash
#git clone https://github.com/broadinstitute/CellBender.git
#pip install -e CellBender --user
cd ../CellBender/examples/remove_background
python generate_tiny_10x_pbmc.py
$HOME/.local/bin/cellbender remove-background      --input ./tiny_raw_gene_bc_matrices/GRCh38      --output ./tiny_10x_pbmc.h5      --expected-cells 500   --total-droplets-included 5000

In [ ]:
import librosa
from scipy.fftpack import fft
import numpy as np
n_fft = 1024
hop_length = 256
n_mels = 40
f_min = 20
f_max = 8000
fmin = 20
fmax = 8000
n_fft = 1024
sample_rate = 16000
sample_rate = 16000
clip, sample_rate = librosa.load('/home/ash022/data/NORSTORE_OSL_DISK/NS9036K/promec/promec/Animesh/HareKrishna.mp3', sr=None)
clip = clip[:132300] # first three seconds of file
n_fft = 1024  # frame length 
start = 45000 # start at a part of the sound thats not silence
x = clip[start:start+n_fft]
X = fft(x, n_fft)
X_magnitude, X_phase = librosa.magphase(X)
X_magnitude_db = librosa.amplitude_to_db(X_magnitude)
stft = librosa.stft(clip, n_fft=n_fft, hop_length=hop_length)
stft_magnitude, stft_phase = librosa.magphase(stft)
stft_magnitude_db = librosa.amplitude_to_db(stft_magnitude)
mel_spec = librosa.feature.melspectrogram(clip, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, sr=sample_rate, power=1.0, fmin=fmin, fmax=fmax)
mel_spec_db = librosa.amplitude_to_db(mel_spec, ref=np.max)

In [ ]:
from deepimpute.deepImpute import deepImpute
import pandas as pd
data = pd.read_csv('../deepimpute/examples/test.csv', index_col=0) # dimension = (cells x genes)
imputed = deepImpute(data, NN_lim='auto', cell_subset=1)
#imputed = deepImpute(data, NN_lim='auto', n_cores=16, cell_subset=1)

In [ ]:
#https://www.analyticsvidhya.com/blog/2019/10/how-to-build-knowledge-graph-text-using-spacy/
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
#print(X_magnitude_db,mel_spec_db)
plt.hist(X_magnitude_db)

In [ ]:
#https://forums.fast.ai/t/transfer-learning-in-fast-ai-how-does-the-magic-work/55620
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend
base_model = keras.applications.resnet_v2.ResNet50V2(weights="imagenet", include_top=False)

for layer in base_model.layers:
    layer.trainable = False

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
mx = keras.layers.GlobalMaxPooling2D()(base_model.output)
out = tf.keras.layers.Concatenate()([avg, mx])
out = keras.layers.BatchNormalization()(out)
out = keras.layers.Dropout(0.25)(out)
out = keras.layers.Dense(512, activation="relu")(out)
out = keras.layers.BatchNormalization()(out)
out = keras.layers.Dropout(0.25)(out)
out = keras.layers.Dense(nr_classes, activation="softmax")(out)

model = keras.models.Model(inputs=base_model.input, outputs=out)

optimizer = tf.keras.optimizers.Adam(lr=0.003)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
#from  rulsif import RULSIF
#from cycler import cycler
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import backend

In [ ]:
import tensorflow as tf
print(tf.__version__)
import datetime
print(datetime.datetime.now())
tf.keras.backend.clear_session()
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print("Eager:",tf.executing_eagerly())
print("GPU:",tf.test.is_gpu_available())#:with tf.device("/gpu:0"):
#tf.keras.backend.clear_session()

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir="..\\notebooks\logs\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train,
          y=y_train,
          epochs=5,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])


In [ ]:
#https://www.kaggle.com/jhoward/some-dicom-gotchas-to-be-aware-of-fastai
#python3 -m pip install torch torchvision feather-format pyarrow --upgrade --user
#python3 -m pip install git+https://github.com/fastai/fastai_dev  --user 
from fastai2.torch_basics      import *
from fastai2.data.all          import *
from fastai2.test              import *
from fastai2.medical.imaging   import *
np.set_printoptions(linewidth=120)

In [ ]:
#https://app.sigopt.com/docs/overview/create
from sigopt import Connection
conn = Connection(client_token="LOPPBTDVPZEWILGCUUZWCNQPHEZTKAGYYFRHGFHLVZUCQJWG")
conn.set_api_url("https://api.sigopt.com")
experiment = conn.experiments().create(
  name="Classifier Accuracy",
  parameters=[
    dict(
      name="gamma",
      bounds=dict(
        min=0,
        max=50
        ),
      type="double"
      )
    ],
  metrics=[
    dict(
      name="Accuracy",
      objective="maximize"
      )
    ],
  observation_budget=30,
  parallel_bandwidth=1,
  project="sigopt-examples"
  )

In [ ]:
#https://eigenfoo.xyz/tests-as-linear/
import pandas as pd
import scipy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
correlated = pd.DataFrame()
correlated["x"] = np.linspace(0, 1)
correlated["y"] = 1.5 * correlated.x + 2 * np.random.randn(len(correlated.x))
scaled = correlated / correlated.std()
r, p = scipy.stats.pearsonr(correlated["x"], correlated["y"])
res1 = smf.ols(formula="y ~ 1 + x", data=correlated).fit()
res2 = smf.ols(formula="y ~ 1 + x", data=scaled).fit()
#print(r,p,"\n",scaled)
plt.plot(scaled)

In [ ]:
import urllib.parse
import urllib.request

url = 'https://www.uniprot.org/uploadlists/'

params = {
'from': 'ACC+ID',
'to': 'ENSEMBL_ID',
'format': 'tab',
'query': 'P40925 P40926 O43175 Q9UM73 P97793'
}

data = urllib.parse.urlencode(params)
data = data.encode('utf-8')
req = urllib.request.Request(url, data)
with urllib.request.urlopen(req) as f:
   response = f.read()
print(response.decode('utf-8'))

In [ ]:
import best
import best.plot
from pymc import MCMC
# This example reproduces Figure 3 of Kruschke, J. (2012) Bayesian estimation supersedes the t test. Journal of Experimental Psychology: General. According to the article, the data were generated from t distributions of known values.
drug = (101,100,102,104,102,97,105,105,98,101,100,123,105,103,100,95,102,106,
        109,102,82,102,100,102,102,101,102,102,103,103,97,97,103,101,97,104,
        96,103,124,101,101,100,101,101,104,100,101)
placebo = (99,101,100,101,102,100,97,101,104,101,102,102,100,105,88,101,100,
           104,100,100,100,101,102,103,97,101,101,100,101,99,101,100,100,
           101,100,99,101,100,102,99,100,99)
data = {'drug':drug,'placebo':placebo}
model = best.make_model( data )
M = MCMC(model)
M.sample(iter=110000, burn=10000)
fig = best.plot.make_figure(M)
fig.savefig('smart_drug.png',dpi=70)

In [ ]:
#https://towardsdatascience.com/coding-ml-tools-like-you-code-ml-models-ddba3357eace
import streamlit as st
x = st.slider('x')
st.write(x, 'squared is', x * x)

In [ ]:
#https://blog.sicara.com/interpretability-deep-learning-models-tensorflow-2-0-7d4ddaa351a3
import numpy as np
import tensorflow as tf
# Layer name to inspect
layer_name = 'block3_conv1'
epochs = 100
step_size = 1.
filter_index = 0
# Create a connection between the input and the target layer
model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)
submodel = tf.keras.models.Model([model.inputs[0]], [model.get_layer(layer_name).output])
# Initiate random noise
input_img_data = np.random.random((1, 224, 224, 3))
input_img_data = (input_img_data - 0.5) * 20 + 128.
# Cast random noise from np.float64 to tf.float32 Variable
input_img_data = tf.Variable(tf.cast(input_img_data, tf.float32))
# Iterate gradient ascents
for _ in range(epochs):
    with tf.GradientTape() as tape:
        outputs = submodel(input_img_data)
        loss_value = tf.reduce_mean(outputs[:, :, :, filter_index])
    grads = tape.gradient(loss_value, input_img_data)
    normalized_grads = grads / (tf.sqrt(tf.reduce_mean(tf.square(grads))) + 1e-5)
    input_img_data.assign_add(normalized_grads * step_size)

In [ ]:
#https://blog.sicara.com/interpretability-deep-learning-models-tensorflow-2-0-7d4ddaa351a3
import numpy as np
import tensorflow as tf
# Layer name to inspect
layer_name = 'block3_conv1'
epochs = 100
step_size = 1.
filter_index = 0
# Create a connection between the input and the target layer
model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)
submodel = tf.keras.models.Model([model.inputs[0]], [model.get_layer(layer_name).output])
# Initiate random noise
input_img_data = np.random.random((1, 224, 224, 3))
input_img_data = (input_img_data - 0.5) * 20 + 128.
# Cast random noise from np.float64 to tf.float32 Variable
input_img_data = tf.Variable(tf.cast(input_img_data, tf.float32))
# Iterate gradient ascents
for _ in range(epochs):
    with tf.GradientTape() as tape:
        outputs = submodel(input_img_data)
        loss_value = tf.reduce_mean(outputs[:, :, :, filter_index])
    grads = tape.gradient(loss_value, input_img_data)
    normalized_grads = grads / (tf.sqrt(tf.reduce_mean(tf.square(grads))) + 1e-5)
    input_img_data.assign_add(normalized_grads * step_size)

In [ ]:
#https://kni.me/w/HJtBHsdkdwo-S64x


In [ ]:
#https://medium.com/dropoutlabs/encrypted-deep-learning-training-and-predictions-with-tf-encrypted-keras-557193284f44
#!pip3 install setuptools --user
!pip3 install tf-encrypted --user

In [ ]:
import tf_encrypted as tfe
from tf_encrypted.keras.losses import BinaryCrossentropy
from tf_encrypted.keras.optimizers import SGD
from common import DataOwner
num_features = 10
batch_size = 100
steps_per_epoch = (training_set_size // batch_size)
epochs = 20
# Provide encrypted training data
data_owner = DataOwner('data-owner', batch_size)
x_train, y_train = data_owner.provide_private_training_data()
# Define model
model = tfe.keras.Sequential()
model.add(tfe.keras.layers.Dense(1, batch_input_shape=[batch_size, num_features]))
model.add(tfe.keras.layers.Activation('sigmoid'))
# Specify optimizer and loss
model.compile(optimizer=SGD(lr=0.01),
              loss=BinaryCrossentropy())
# Start training
model.fit(x_train,
          y_train,
          epochs=epochs,
          steps_per_epoch=steps_per_epoch)

In [ ]:
import numpy as np
import ipyvolume as ipv
V = np.zeros((128,128,128)) # our 3d array
# outer box
V[30:-30,30:-30,30:-30] = 0.75
V[35:-35,35:-35,35:-35] = 0.0
# inner box
V[50:-50,50:-50,50:-50] = 0.25
V[55:-55,55:-55,55:-55] = 0.0
ipv.quickvolshow(V, level=[0.25, 0.75], opacity=0.03, level_width=0.1, data_min=0, data_max=1)

In [ ]:
#https://pytorch.org/docs/stable/jit.html#mixing-tracing-and-scripting
import torch
def foo(x, y):
    return 2 * x + y
traced_foo = torch.jit.trace(foo, (torch.rand(3), torch.rand(3)))
traced_foo
#@torch.jit.script
#def bar(x):
#    return traced_foo(x, x)

In [ ]:
from catboost import CatBoostClassifier, Pool
train_data = [[1,3], [0,4], [1,7], [0,3]]
train_labels = [1,0,1,1]
catboost_pool = Pool(train_data, train_labels)
model = CatBoostClassifier(learning_rate=0.03)
model.fit(train_data, train_labels, verbose=False, plot=True)

In [ ]:
!ann_solo HumanPlasma_2012-08_all.splib  ./RawRead/171010_Ip_Hela_ugi.raw.intensity0.charge0.MGF  out_filename --precursor_tolerance_mass 10 --precursor_tolerance_mode ppm --fragment_mz_tolerance 10
!head -n 2 out_filename.mztab
!tail -n 2 out_filename.mztab

In [ ]:
import faiss, numpy
vals=[faiss.Kmeans(10, 20).train(numpy.random.rand(1000, 10).astype('float32')) for i in range(1000)]

In [ ]:
#print(vals)
import matplotlib.pyplot as plt
plt.hist(vals)

In [ ]:
#https://towardsdatascience.com/audio-classification-using-fastai-and-on-the-fly-frequency-transforms-4dbe1b540f89
import librosa
from scipy.fftpack import fft
n_fft = 1024
hop_length = 256
n_mels = 40
f_min = 20
f_max = 8000
fmin = 20
fmax = 8000
n_fft = 1024
sample_rate = 16000
sample_rate = 16000
clip, sample_rate = librosa.load('/home/ash022/.conda/pkgs/scipy-1.1.0-py36he2b7bc3_2/lib/python3.6/site-packages/scipy/io/tests/data/test-44100Hz-le-1ch-4bytes.wav', sr=None)
clip = clip[:132300] # first three seconds of file
n_fft = 1024  # frame length 
start = 45000 # start at a part of the sound thats not silence
x = clip[start:start+n_fft]
X = fft(x, n_fft)
X_magnitude, X_phase = librosa.magphase(X)
X_magnitude_db = librosa.amplitude_to_db(X_magnitude)
stft = librosa.stft(clip, n_fft=n_fft, hop_length=hop_length)
stft_magnitude, stft_phase = librosa.magphase(stft)
stft_magnitude_db = librosa.amplitude_to_db(stft_magnitude)
mel_spec = librosa.feature.melspectrogram(clip, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, sr=sample_rate, power=1.0, fmin=fmin, fmax=fmax)
mel_spec_db = librosa.amplitude_to_db(mel_spec, ref=np.max)
NSYNTH_IMAGES = '.'
instrument_family_pattern = r'(\w+)_\w+_\d+-\d+-\d+.png$'
data = (ImageItemList.from_folder(NSYNTH_IMAGES)
          .split_by_folder()
          .label_from_re(instrument_family_pattern)
          .databunch())
learn = create_cnn(data, models.resnet18, metrics=accuracy)
learn.fit_one_cycle(3)
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(10, 10), dpi=60)
#https://pytorch.org/docs/master/torch.html#torch.stft

In [ ]:
import vaex
ds = vaex.example() 
import numpy as np
# creates an expression (nothing is computed)
r = np.sqrt(ds.x**2 + ds.y**2 + ds.z**2)
r  # for convenience, we print out some values
#ds.mean(ds.r, binby=ds.x, shape=32, limits=[-10, 10]) # create statistics on a regular grid (1d)
xcounts = ds.count(binby=ds.x, limits=[-10, 10], shape=64)
print(xcounts)

In [ ]:
#https://github.com/llSourcell/Gaussian_Mixture_Models/blob/master/intro_to_gmm_%26_em.ipynb
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import seaborn as sns
sns.set_style("white")
x = np.linspace(start=-10, stop=10, num=1000)
y = stats.norm.pdf(x, loc=0, scale=1.5)+stats.norm.pdf(x, loc=1.5, scale=.5) 
#%matplotlib inlinex = np.linspace(start=-10, stop=10, num=1000)
#plt.plot(x, y)
sns.distplot(y)

In [ ]:
#https://threader.app/thread/1105139360226140160
import tensorflow as tf
print(tf.__version__)
import datetime
print(datetime.datetime.now())
tf.keras.backend.clear_session() 
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print("Eager:",tf.executing_eagerly())
print("GPU:",tf.test.is_gpu_available())#:with tf.device("/gpu:0"):
#tf.keras.backend.clear_session()

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir="/mnt/promec-ns9036k/.tools/exciting-wildebeest-tensorboard/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train,
          y=y_train,
          epochs=5,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

In [ ]:
from tensorboard import summary
from tensorboard.plugins.custom_scalar import layout_pb2
layout_summary = summary_lib.custom_scalar_pb(layout_pb2.Layout(
  category=[
    layout_pb2.Category(
      title='losses',
      chart=[
          layout_pb2.Chart(
              title='losses',
              multiline=layout_pb2.MultilineChartContent(
                tag=[r'loss.*'],
              )),
          layout_pb2.Chart(
              title='baz',
              margin=layout_pb2.MarginChartContent(
                series=[
                  layout_pb2.MarginChartContent.Series(
                    value='loss/baz/scalar_summary',
                    lower='baz_lower/baz/scalar_summary',
                    upper='baz_upper/baz/scalar_summary'),
                ],
              )),
      ]),
    layout_pb2.Category(
      title='trig functions',
      chart=[
          layout_pb2.Chart(
              title='wave trig functions',
              multiline=layout_pb2.MultilineChartContent(
                tag=[r'trigFunctions/cosine', r'trigFunctions/sine'],
              )),
          # The range of tangent is different. Let's give it its own chart.
          layout_pb2.Chart(
              title='tan',
              multiline=layout_pb2.MultilineChartContent(
                tag=[r'trigFunctions/tangent'],
              )),
      ],
      # This category we care less about. Let's make it initially closed.
      closed=True),
  ]))
writer.add_summary(layout_summary)

In [ ]:
#https://www.nature.com/articles/s41596-018-0073-y A comprehensive evaluation of popular proteomics software workflows for label-free proteome quantification and imputation tensorly.decomposition.robust_pca
!pip install bbknn --user #kBET
#SCRAN pooling with ComBat9


In [ ]:
import bbknn
#bbknn.bbknn(X2[:,0])
#https://nbviewer.jupyter.org/github/Teichlab/bbknn/blob/master/examples/pancreas.ipynb
distances, connectivities = bbknn.bbknn_pca_matrix(X2, X2>10)

In [ ]:
#plt.plot(distances)
print(connectivities)#.shape()

In [ ]:
#https://towardsdatascience.com/audio-classification-using-fastai-and-on-the-fly-frequency-transforms-4dbe1b540f89
clip, sample_rate = librosa.load(filename, sr=None)
clip = clip[:132300] # first three seconds of file
n_fft = 1024  # frame length 
start = 45000 # start at a part of the sound thats not silence
x = clip[start:start+n_fft]
X = fft(x, n_fft)
X_magnitude, X_phase = librosa.magphase(X)
X_magnitude_db = librosa.amplitude_to_db(X_magnitude)
stft = librosa.stft(clip, n_fft=n_fft, hop_length=hop_length)
stft_magnitude, stft_phase = librosa.magphase(stft)
stft_magnitude_db = librosa.amplitude_to_db(stft_magnitude)
mel_spec = librosa.feature.melspectrogram(clip, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, sr=sample_rate, power=1.0, fmin=fmin, fmax=fmax)
mel_spec_db = librosa.amplitude_to_db(mel_spec, ref=np.max)
n_fft = 1024
hop_length = 256
n_mels = 40
f_min = 20
f_max = 8000
sample_rate = 16000
NSYNTH_IMAGES = 'data/nsynth_acoustic_images'
instrument_family_pattern = r'(\w+)_\w+_\d+-\d+-\d+.png$'
data = (ImageItemList.from_folder(NSYNTH_IMAGES)
          .split_by_folder()
          .label_from_re(instrument_family_pattern)
          .databunch())
learn = create_cnn(data, models.resnet18, metrics=accuracy)
learn.fit_one_cycle(3)
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(10, 10), dpi=60)
#https://pytorch.org/docs/master/torch.html#torch.stft

In [ ]:
#https://github.com/llSourcell/Gaussian_Mixture_Models/blob/master/intro_to_gmm_%26_em.ipynb
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import seaborn as sns
sns.set_style("white")
x = np.linspace(start=-10, stop=10, num=1000)
y = stats.norm.pdf(x, loc=0, scale=1.5)+stats.norm.pdf(x, loc=1.5, scale=.5) 
#%matplotlib inlinex = np.linspace(start=-10, stop=10, num=1000)
plt.plot(x, y)
sns.distplot(y)

In [ ]:
class Gaussian:
    "Model univariate Gaussian"
    def __init__(self, mu, sigma):
        #mean and standard deviation
        self.mu = mu
        self.sigma = sigma

    #probability density function
    def pdf(self, datum):
        "Probability of a data point given the current parameters"
        u = (datum - self.mu) / abs(self.sigma)
        y = (1 / (sqrt(2 * pi) * abs(self.sigma))) * exp(-u * u / 2)
        return y
    #printing model values
    def __repr__(self):
        return 'Gaussian({0:4.6}, {1:4.6})'.format(self.mu, self.sigma)

In [ ]:
data
best_single = Gaussian(np.mean(data), np.std(data))
print('Best single Gaussian: μ = {:.2}, σ = {:.2}'.format(best_single.mu, best_single.sigma))

In [ ]:
#https://github.com/cc-skuehn/Manifold_Learning/blob/master/Manifold_tSNE_UMAP_Advanced.ipynb
print(__doc__)

from time import time

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
%matplotlib inline
from sklearn import manifold, datasets
import numpy as np

# Next line to silence pyflakes. This import is needed.
Axes3D

# Parameters
n_points = 1000
n_components = 2 # for tSNE / PCA
init_random_state=0 # for all methods, change if you like (42 is the value of choice)

# Generate a random point cloud, take z component for coloring
X2 = np.random.rand(n_points, 3)
color2 = X2[:,2]

fig = plt.figure(figsize=(12, 12))
plt.suptitle("Random Noise %i points"
             % (n_points), fontsize=24)

ax = fig.add_subplot(111, projection='3d')
ax.scatter(X2[:, 0], X2[:, 1], X2[:, 2], c=color2, cmap=plt.cm.Spectral,s=125)
#ax.view_init(4, -72)

In [ ]:
from sklearn import random_projection
from sklearn.decomposition import PCA

fig = plt.figure(figsize=(16, 8))
plt.suptitle("PCA and Random Projections with %i points, %i components"
             % (n_points, n_components), fontsize=14)

# PCA
pca = PCA(n_components=2)
Y = pca.fit_transform(X2)

ax = fig.add_subplot(131)
plt.scatter(Y[:, 0], Y[:, 1], c=color2, cmap=plt.cm.Spectral)
plt.title("%s " % ('PCA'))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')

# Gaussian Random Projection - GRP
transformer = random_projection.GaussianRandomProjection(n_components=2, random_state=init_random_state)
Y = transformer.fit_transform(X2)
    
ax = fig.add_subplot(132)
plt.scatter(Y[:, 0], Y[:, 1], c=color2, cmap=plt.cm.Spectral)
plt.title("%s " % ('GRP'))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')

# Sparse Random ProjectiSRP
transformer = random_projection.SparseRandomProjection(n_components=2, random_state=init_random_state)
Y = transformer.fit_transform(X2)
    
ax = fig.add_subplot(133)
plt.scatter(Y[:, 0], Y[:, 1], c=color2, cmap=plt.cm.Spectral)
plt.title("%s " % ('SRP'))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')

In [ ]:
t0 = time()
tsne = manifold.TSNE(n_components=2, random_state=1, perplexity=10)
Y = tsne.fit_transform(X2)
t1 = time()

In [ ]:
fig = plt.figure(figsize=(12, 12))
plt.suptitle("t-SNE applied to Random Noise with %i points"
             % (n_points), fontsize=14)

ax = fig.add_subplot(111)
plt.scatter(Y[:, 0], Y[:, 1], c=color2, cmap=plt.cm.Spectral)
plt.title("t-SNE (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')

In [ ]:
import umap
t0 = time()
Y= umap.UMAP(n_neighbors=15,
                      min_dist=0.3,
                      metric='euclidean').fit_transform(X2)
t1 = time()
fig = plt.figure(figsize=(12, 12))
plt.suptitle("UMAP applied to Random Noise with %i points"
             % (n_points), fontsize=14)

ax = fig.add_subplot(111)
plt.scatter(Y[:, 0], Y[:, 1], c=color2, cmap=plt.cm.Spectral)
plt.title("UMAP (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')

In [ ]:
#!pip install biopython --user
!pip install umap --user

In [ ]:
#https://towardsdatascience.com/extreme-rare-event-classification-using-autoencoders-in-keras-a565b386f098
#https://github.com/cran2367/autoencoder_classifier/blob/master/autoencoder_classifier.ipynb
#https://github.com/Microsoft/dowhy


In [ ]:
import Bio
from Bio.Seq import Seq
seq = Seq("AGTACACTGGT")
print(seq.complement()[::-1], seq.reverse_complement())

In [ ]:
%%perl
$revseq=reverse("AGTACACTGGT");#=~tr/ATCG/TAGC/;
#reverse($revseq)=~tr/ATCG/TAGC/;
print $revseq,"\n";

In [ ]:
from Bio.Seq import Seq
fasta = Seq("AGTACACTGGT")
SEQ_LEN = len(fasta)
rowStrand = 'Rev'
rowStart=len(fasta)
seq = fasta[end-SEQ_LEN:end].__str__()
if rowStrand == 'Rev':
  end = rowStart
  seq = fasta[end-SEQ_LEN:end].complement().__str__()
seq

In [ ]:
#https://github.com/verilylifesciences/variant-annotation/blob/master/interactive/InteractiveVariantAnnotation.ipynb
#https://www.ensembl.org/info/docs/tools/vep/vep_formats.html#input

%%bq query
#standardSQL
  --
  -- Return variants for sample NA12878 that are:
  --   annotated as 'pathogenic' or 'other' in ClinVar
  --   with observed population frequency less than 5%
  --
  WITH sample_variants AS (
  SELECT
    -- Remove the 'chr' prefix from the reference name.
    REGEXP_EXTRACT(reference_name, r'chr(.+)') AS chr,
    start,
    reference_bases,
    alt,
    call.call_set_name
  FROM
    `genomics-public-data.platinum_genomes_deepvariant.single_sample_genome_calls` v,
    v.call call,
    v.alternate_bases alt WITH OFFSET alt_offset
  WHERE
    call_set_name = 'NA12878_ERR194147'
    -- Require that at least one genotype matches this alternate.
    AND EXISTS (SELECT gt FROM UNNEST(call.genotype) gt WHERE gt = alt_offset+1)
    ),
  --
  --
  rare_pathenogenic_variants AS (
  SELECT
    -- ClinVar does not use the 'chr' prefix for reference names.
    reference_name AS chr,
    start,
    reference_bases,
    alt,
    CLNHGVS,
    CLNALLE,
    CLNSRC,
    CLNORIGIN,
    CLNSRCID,
    CLNSIG,
    CLNDSDB,
    CLNDSDBID,
    CLNDBN,
    CLNREVSTAT,
    CLNACC
  FROM
    `bigquery-public-data.human_variant_annotation.ncbi_clinvar_hg38_20170705` v,
    v.alternate_bases alt
  WHERE
    -- Variant Clinical Significance, 0 - Uncertain significance, 1 - not provided,
    -- 2 - Benign, 3 - Likely benign, 4 - Likely pathogenic, 5 - Pathogenic,
    -- 6 - drug response, 7 - histocompatibility, 255 - other
    EXISTS (SELECT sig FROM UNNEST(CLNSIG) sig WHERE REGEXP_CONTAINS(sig, '(4|5|255)'))
    -- TRUE if >5% minor allele frequency in 1+ populations
    AND G5 IS NULL
)
 --
 --
SELECT
  *
FROM
  sample_variants
JOIN
  rare_pathenogenic_variants USING(chr,
    start,
    reference_bases,
    alt)
ORDER BY
  chr,
  start,
  reference_bases,
  alt

In [ ]:
from pathlib import Path
home=Path.home()
pathFiles = home / Path('promec/Elite/LARS/2017/april')
fileName='evidence.txt'
trainList=list(pathFiles.rglob(fileName))
print(pathFiles,trainList)

In [ ]:
import pandas as pd
df=pd.DataFrame()
for f in trainList:
    peptideHits=pd.read_csv(f,low_memory=False,sep='\t')
    print(f)
    peptideHits['Name']=f
    df=pd.concat([df,peptideHits],sort=False)
#print(df.head(1))
#print(df.columns)
colStrName="Raw file"
print(df.columns.get_loc(colStrName))
dfDP=df.loc[:, df.columns.str.startswith(colStrName)]
dfDP=dfDP[dfDP[colStrName].notnull()]
#dfDP=dfDP.rename(columns = lambda x : str(x)[3:])
dfDPcnt=dfDP[colStrName].value_counts()
dfDPcnt[dfDPcnt>10].plot(kind='barh')

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline

import pandas as pd

from scipy.optimize import curve_fit as cf
import ipywidgets
ipywidgets.__version__
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
def f(x):
    return x
interact(f, x=10);

In [ ]:
N_samples = 25
x=np.linspace(-2,2,N_samples)

In [ ]:
def f(x,a,mu,sigma):
    r=a*np.exp(-(x-mu)**2/(2*sigma**2))
    return (r)

In [ ]:
def func(amplitude,ideal_mu,ideal_sigma,noise_sd,noise_mean):
    r=amplitude*np.exp(-(x-ideal_mu)**2/(2*ideal_sigma**2))
    plt.figure(figsize=(8,5))
    plt.plot(x,r,c='k',lw=3)
    r= r+np.random.normal(loc=noise_mean,scale=noise_sd,size=N_samples)
    plt.scatter(x,r,edgecolors='k',c='yellow',s=60)
    plt.grid(True)
    plt.show()
    return (r)

In [ ]:
y=interactive(func,amplitude=[1,2,3,4,5],ideal_mu=(-5,5,0.5),
              ideal_sigma=(0,2,0.2),
              noise_sd=(0,1,0.1),noise_mean=(-1,1,0.2))
display(y

In [ ]:
# Curve-fit optimizer
p = y.result
p1,_=cf(f,xdata=x,ydata=p)
par_lst=list(p1)

# Gather ideal parameters from the interactive widget object
p2=list(y.kwargs.values())
p3=p2[0:3]

# Make the data matrix
data = np.array([p3,par_lst])

# Put it in a Data Frame
df=pd.DataFrame(data=data.T,index=['Amplitude','Mean','Std.Dev'],columns=['Original Params','Estimated Params'])
print(df)

# Plot the ideal and estimated curves
plt.figure(figsize=(10,6))
plt.plot(x,f(x,par_lst[0],par_lst[1],par_lst[2]),'k--',lw=2)
plt.plot(x,f(x,p3[0],p3[1],p3[2]),c='green',lw=2)
plt.legend(['Fitted curve with noise','Ideal curve w/o noise'],fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
%%perl
$c=0;
while($c<10){$c++;@b=qw/A T G C/;print $b[int(rand(4))]}

In [ ]:
%load_ext rpy2.ipython
#plot(sin(1:100))# Hide warnings if there are any
import warnings
warnings.filterwarnings('ignore')
# Load in the r magic
# We need ggplot2
%R require(ggplot2)
# Load in the pandas library
import pandas as pd 
# Make a pandas DataFrame
df = pd.DataFrame({'Alphabet': ['a', 'b', 'c', 'd','e', 'f', 'g', 'h','i'],
                   'A': [4, 3, 5, 2, 1, 7, 7, 5, 9],
                   'B': [0, 4, 3, 6, 7, 10,11, 9, 13],
                   'C': [1, 2, 3, 1, 2, 3, 1, 2, 3]})
# Take the name of input variable df and assign it to an R variable of the same name

df

In [ ]:
%%R
require(ggplot2)
plot(sin(1:100))
ggplot(x=sin(1:100),y=(1:100))

In [ ]:
%%perl
use Statistics::R;
  my $R = Statistics::R->new();
  my $output_file = "file.ps";
  $R->run(qq`postscript("$output_file", horizontal=FALSE, width=500, height=500)`);
  $R->run(q`plot(c(1, 5, 10), type = "l")`);
  $R->run(q`dev.off()`);
  my $input_value = 5;
  $R->set('x', $input_value);
  $R->run(q`y <- x^2`);
  my $output_value = $R->get('y');
  print "y = $output_value\n";
  $R->stop();


In [ ]:
#https://github.com/bloomberg/bqplot/blob/master/examples/Applications/Outlier%20Detection.ipynb
from scipy.stats import percentileofscore
from scipy.interpolate import interp1d
import bqplot.pyplot as plt
from bqplot import *
from traitlets import List, Float, observe
from ipywidgets import IntRangeSlider, Layout, VBox, HBox, jslink
from pandas import DatetimeIndex
import numpy as np
import pandas as pd

def quantile_space(x, q1=0.1, q2=0.9):
    '''
    Returns a function that squashes quantiles between q1 and q2
    '''
    q1_x, q2_x = np.percentile(x, [q1, q2])
    qs = np.percentile(x, np.linspace(0, 100, 100))
    def get_quantile(t):
        return np.interp(t, qs, np.linspace(0, 100, 100))
    def f(y):
        return np.interp(get_quantile(y), [0, q1, q2, 100], [-1, 0, 0, 1])
    return f

class DNA(VBox):
    
    colors = List()
    q1 = Float()
    q2 = Float()
    
    def __init__(self, data, **kwargs):
        self.data = data
        date_x, date_y = False, False
        transpose = kwargs.pop('transpose', False)
        if transpose is True:
            if type(data.index) is DatetimeIndex:
                self.x_scale = DateScale()
            if type(data.columns) is DatetimeIndex:
                self.y_scale = DateScale()
            x, y = list(data.columns.values), data.index.values
        else:
            if type(data.index) is DatetimeIndex:
                date_x = True
            if type(data.columns) is DatetimeIndex:
                date_y = True
            x, y = data.index.values, list(data.columns.values)
            
        self.q1, self.q2 = kwargs.pop('quantiles', (1, 99))
        
        self.quant_func = quantile_space(self.data.values.flatten(), q1=self.q1, q2=self.q2)
        self.colors = kwargs.pop('colors', ['Red', 'Black', 'Green'])
        
        self.x_scale = DateScale() if date_x is True else LinearScale()
        self.y_scale = DateScale() if date_y is True else OrdinalScale(padding_y=0)
        self.color_scale = ColorScale(colors=self.colors)
        self.heat_map = HeatMap(color=self.quant_func(self.data.T), x=x, y=y, scales={'x': self.x_scale, 'y': self.y_scale,
                                                                               'color': self.color_scale})
        self.x_ax = Axis(scale=self.x_scale)
        self.y_ax = Axis(scale=self.y_scale, orientation='vertical')
        show_axes = kwargs.pop('show_axes', True)
        self.axes = [self.x_ax, self.y_ax] if show_axes is True else []
        
        self.height = kwargs.pop('height', '800px')
        self.layout = kwargs.pop('layout', Layout(width='100%', height=self.height, flex='1'))
        self.fig_margin = kwargs.pop('fig_margin', {'top': 60, 'bottom': 60, 'left': 150, 'right': 0})
        kwargs.setdefault('padding_y', 0.0)
        
        self.create_interaction(**kwargs)
        
        self.figure = Figure(marks=[self.heat_map], axes=self.axes, fig_margin=self.fig_margin, 
                             layout=self.layout, min_aspect_ratio=0.,**kwargs)
        
        super(VBox, self).__init__(children=[self.range_slider, self.figure], layout=Layout(align_items='center',
                                                                                           width='100%',
                                                                                           height='100%'),
                                   **kwargs)
        
    def create_interaction(self, **kwargs):
        self.range_slider = IntRangeSlider(description='Filter Range', value=(self.q1, self.q2), layout=Layout(width='100%'))
        self.range_slider.observe(self.slid_changed, 'value')
        self.observe(self.changed, ['q1', 'q2'])
        
        
    def slid_changed(self, new):
        self.q1 = self.range_slider.value[0]
        self.q2 = self.range_slider.value[1]
        
    def changed(self, new):
        self.range_slider.value = (self.q1, self.q2)
        
        self.quant_func = quantile_space(self.data.values.flatten(), q1=self.q1, q2=self.q2)
        self.heat_map.color = self.quant_func(self.data.T)
        
    def get_filtered_df(self, fill_type='median'):
        q1_x, q2_x = np.percentile(self.data, [self.q1, self.q2])
        if fill_type == 'median':
            return self.data[(self.data >= q1_x) & (self.data <= q2_x)].apply(lambda x: x.fillna(x.median()))
        elif fill_type == 'mean':
            return self.data[(self.data >= q1_x) & (self.data <= q2_x)].apply(lambda x: x.fillna(x.mean()))
        else:
            raise ValueError("fill_type must be one of ('median', 'mean')")

In [ ]:
size = 100
def num_to_col_letters(num):
    letters = ''
    while num:
        mod = (num - 1) % 26
        letters += chr(mod + 65)
        num = (num - 1) // 26
    return ''.join(reversed(letters))

letters = []

for i in range(1, size+1):
    letters.append(num_to_col_letters(i))

data = pd.DataFrame(np.random.randn(size, size), columns=letters)

data_dna = DNA(data, title='DNA of our Data', height='1400px', colors=['Red', 'White', 'Green'])
data_dna

In [ ]:
data_dna.q1, data_dna.q2 = 5, 95
data_clean = data_dna.get_filtered_df()
data_mean = data_dna.get_filtered_df(fill_type='mean')
DNA(data_clean, title='Cleaned Data', height='1200px', colors=['Red', 'White', 'Green'])

In [ ]:
#https://research.googleblog.com/2017/10/eager-execution-imperative-define-by.html?utm_campaign=devshow_series_eagerexecutionfortensorflow_111017&utm_source=gdev&utm_medium=yt-desc
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()


In [ ]:
print(tf.__version__)

In [ ]:
x = [[2.]]
m = tf.matmul(x, x)
print(m)

In [ ]:
a = tf.constant(25)
counter = 0
while not tf.equal(a, 1):
  if tf.equal(a % 2, 0):
    a = a / 2
  else:
    a = 3 * a + 1
  print(a)

In [ ]:
def square(x):
  return tf.multiply(x, x)

grad = tfe.gradients_function(square)

print(square(3.))    # [9.]
print(grad(3.))      # [6.]

In [ ]:
gradgrad = tfe.gradients_function(lambda x: grad(x)[0])

print(gradgrad(3.))  # [2.]

In [ ]:
def abs(x):
  return x if x > 0. else -x

grad = tfe.gradients_function(abs)

print(grad(2.0))  # [1.]
print(grad(-2.0)) # [-1.]

In [ ]:
def log1pexp(x):
  return tf.log(1 + tf.exp(x))
grad_log1pexp = tfe.gradients_function(log1pexp)

# The gradient computation works fine at x = 0.
print(grad_log1pexp(0.))
# [0.5]
# However it returns a `nan` at x = 100 due to numerical instability.
print(grad_log1pexp(100.))
# [nan]

In [ ]:
@tfe.custom_gradient
def log1pexp(x):
  e = tf.exp(x)
  def grad(dy):
    return dy * (1 - 1 / (1 + e))
  return tf.log(1 + e), grad
grad_log1pexp = tfe.gradients_function(log1pexp)

# Gradient at x = 0 works as before.
print(grad_log1pexp(0.))
# [0.5]
# And now gradient computation at x=100 works as well.
print(grad_log1pexp(100.))
# [1.0]

In [ ]:
class MNISTModel(tfe.Network):
  def __init__(self):
    super(MNISTModel, self).__init__()
    self.layer1 = self.track_layer(tf.layers.Dense(units=10))
    self.layer2 = self.track_layer(tf.layers.Dense(units=10))
  def call(self, input):
    """Actually runs the model."""
    result = self.layer1(input)
    result = self.layer2(result)
    return result

In [ ]:
# Let's make up a blank input image
model = MNISTModel()
batch = tf.zeros([1, 1, 784])
print(batch.shape)
# (1, 1, 784)
result = model(batch)
print(result)
# tf.Tensor([[[ 0.  0., ...., 0.]]], shape=(1, 1, 10), dtype=float32)

In [ ]:
def loss_function(model, x, y):
  y_ = model(x)
  return tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_) 

#And then, our training loop:

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
for (x, y) in tfe.Iterator(dataset):
  grads = tfe.implicit_gradients(loss_function)(model, x, y)
  optimizer.apply_gradients(grads)

    #implicit_gradients() 

    #calculates the derivatives of loss_function with respect to all the TensorFlow variables used during its computation.
#We can move computation to a GPU the same way we’ve always done with TensorFlow:
with tf.device("/gpu:0"):
  for (x, y) in tfe.Iterator(dataset):
    optimizer.minimize(lambda: loss_function(model, x, y))